<a href="https://colab.research.google.com/github/Xixaus/Nastroje/blob/main/CD_extrakce%2Bvyhlazen%C3%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Pro spuštění skriptu je nutné se přihlásit do Google účtu.
2. Spusťte 1. buňku tlačítkem ▶ v levém horním rohu nebo kliknutím do buňky a stistnutím klávesové zkratky ctrl+enter aby se načetly knihovny a vytvořili složky.
3. Do složky Samples vložte soubory s vzorky a do složky blanks vložte soubory s blanky

    !!! Je nutné aby pH vzoku bylo v názvu v souboru v následujícím formátu: ... _ 7_05_pufr.txt, kdy tímto způsobem bude končit název souboru !!!

3. Spusťte 2. buňku pro vygenerování grafu a souboru
4. Soubor data.txt vložte do Bestselu, pokud Vám bude hlásit chybu, zkopírujte všechny data ze souboru a vložte do okna v Bestselu

In [ ]:
#načtení knihoven
import os
import shutil
import pandas as pd
from scipy.signal import savgol_filter
import plotly.graph_objects as go

#vytvoření potřebných složek pro data
os.makedirs("/content/samples", exist_ok=True)
os.makedirs("/content/blanks", exist_ok=True)

#smazání zbytečné složky
if os.path.exists("/content/sample_data"):
    shutil.rmtree("/content/sample_data")

In [ ]:
#funkce pro extrakci dat ze souborů
def extract_data(folder_path):
    file_list = os.listdir(folder_path)
    df = pd.DataFrame() #vytvoření prázného dataframe

    for filename in file_list: #iterace každého souboru ve složce
        if filename.endswith(".txt"): #podmínka pro extrakci pouze z .txt souborů
            number = (filename.split("_")[-3].split(".")[0]) + "." + filename.split("_")[-2].split(".")[0] #pojmenování spektra, konkrétně pH, které je na konci ve formátu ..._7_05_pufr.txt

            axis = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[0], skiprows=19, header=None) #extrakce vlnových délek
            axis = axis.values.flatten()

            data = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[1], skiprows=19, header=None) # extrakce spektra
            data = data.rename(columns={1: float(number)}) #přejmenování spektra podle promněnné number
            data.index = axis

            df = pd.concat([df, data], axis=1) #spojení dataframe se spectrem

    df = df.sort_index(axis=1) #seřazení sepekter podle názvu (number)
    return(df)

samples = extract_data("/content/samples") #extrakce naměřených spekter
blanks = extract_data("/content/blanks") #extrakce blanků

spectra = samples - blanks #odečtení blanků od spekter

for column in spectra.columns:
    spectra[column] = savgol_filter(spectra[column], window_length=7, polyorder=2, deriv=0) #vyhlazení pomocí savitzkého-golayových filtrů

fig = go.Figure() #vykreslení grafu

for column in spectra.columns:
    fig.add_trace(go.Scatter(y=spectra[column], x=spectra.index, mode='lines', name=str(column)))

fig.show()

spectra.to_csv('data.txt', sep='\t') #uložení spekter do .txt souboru